# The Injury Risk Predictor

This script predicts the injury risk of MLB players based on their game logs and injury history using a pre-trained XGBoost model.
It includes the following steps:

1. Load game logs and injury data from a database.
2. Preprocess the data to create features for the model.
3. Load the pre-trained model.
4. Predict injury risk probabilities for current players.

## Imports 

In [1]:

# Import necessary libraries

import joblib
import pandas as pd
import numpy as np
import sqlite3
from datetime import timedelta
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE 
from xgboost import plot_importance
from datetime import timedelta

db_path = '/Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr/mlb/mlb_players.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

## Create Game Logs Df

In [31]:
mlb_player_stats = pd.read_sql_query("SELECT * FROM mlb_player_stats", conn)
mlb_player_stats.columns

Index(['mlb_player_id', 'stat_type', 'stat_group', 'season', 'game_date',
       'team_id', 'team_name', 'opponent_id', 'opponent_name', 'position',
       'gamesPlayed', 'games', 'gamesStarted', 'assists', 'putOuts', 'errors',
       'chances', 'fielding', 'doublePlays', 'triplePlays', 'throwingErrors',
       'rangeFactorPerGame', 'rangeFactorPer9Inn', 'innings', 'inningsPitched',
       'catcherERA', 'flyOuts', 'groundOuts', 'airOuts', 'passedBall', 'wins',
       'losses', 'wildPitches', 'pickoffs', 'runs', 'doubles', 'triples',
       'homeRuns', 'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits',
       'hitByPitch', 'avg', 'atBats', 'obp', 'slg', 'ops', 'caughtStealing',
       'stolenBases', 'stolenBasePercentage', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'numberOfPitches', 'plateAppearances',
       'totalBases', 'rbi', 'leftOnBase', 'sacBunts', 'sacFlies', 'babip',
       'groundOutsToAirouts', 'catchersInterference', 'atBatsPerHomeRun',
       'summary', '

mlb_player_stats.columns

In [66]:

# Game Logs for all players in the MLB 

query = """
SELECT mlb_player_stats.* , mlb_player_info.*
FROM mlb_player_stats
INNER JOIN mlb_player_info ON mlb_player_stats.mlb_player_id = mlb_player_info.mlb_player_id
WHERE stat_type = 'gameLog'
"""

game_log_pre = pd.read_sql_query(query, conn)

game_log_pre = game_log_pre.loc[:, ~game_log_pre.columns.duplicated()]

selected_columns = [
    # Identity and basic game info
    'fullName', 'mlb_player_id', 'game_date', 'team_name', 'opponent_name',

    # Workload & fatigue
    'gamesPlayed', 'gamesStarted', 'innings', 'plateAppearances', 'numberOfPitches',
    'game_number', 'atBatsPerHomeRun',

    # Performance (extended)
    'atBats', 'runs', 'hits', 'totalBases', 'doubles', 'triples', 'homeRuns', 'rbi',
    'baseOnBalls', 'intentionalWalks', 'strikeOuts', 'stolenBases', 'caughtStealing',
    'hitByPitch', 'sacBunts', 'sacFlies', 'avg', 'obp', 'slg',

    # Performance trends / condition indicators
    #'groundIntoDoublePlay', 'leftOnBase',

    # Positional & biomechanical stress factors
     'primaryPosition', 'pitchHand', 'batSide',

    # Biometric and career timeline
    'height', 'weight', 'currentAge', 'birthDate', 'debutDate'
]

game_logs_df = game_log_pre[selected_columns]

# Fill rows with missing values
game_logs_df = game_logs_df.fillna(0)

game_logs_df = game_logs_df.rename(columns={
    # Identity and team info
    'fullName': 'Name',
    'mlb_player_id': 'PlayerID',
    'game_date': 'Date',
    'team_name': 'Team',
    'opponent_name': 'OPP',

    # Workload & fatigue
    'gamesPlayed': 'GP',
    'gamesStarted': 'GS',
    'innings': 'INN',
    'plateAppearances': 'PA',
    'numberOfPitches': 'NP',
    'game_number': 'Game#',
    'atBatsPerHomeRun': 'AB/HR',

    # Batting performance
    'atBats': 'AB',
    'runs': 'R',
    'hits': 'H',
    'totalBases': 'TB',
    'doubles': '2B',
    'triples': '3B',
    'homeRuns': 'HR',
    'rbi': 'RBI',
    'baseOnBalls': 'BB',
    'intentionalWalks': 'IBB',
    'strikeOuts': 'SO',
    'stolenBases': 'SB',
    'caughtStealing': 'CS',
    'hitByPitch': 'HBP',
    'sacBunts': 'SAC',
    'sacFlies': 'SF',
    'avg': 'AVG',
    'obp': 'OBP',
    'slg': 'SLG',

    # Performance trends
    'groundIntoDoublePlay': 'GIDP',
    'leftOnBase': 'LOB',

    # Positional & biomechanics
    'position': 'POS',
    'primaryPosition': 'PrimaryPOS',
    'pitchHand': 'Throw',
    'batSide': 'Bat',

    # Biometric & career data
    'height': 'Height',
    'weight': 'Weight',
    'currentAge': 'Age',
    'birthDate': 'BirthDate',
    'debutDate': 'Debut'
})

# STEP 1: Load data
game_logs_df.rename(columns={"PlayerID": "mlb_player_id"}, inplace=True)
game_logs = game_logs_df # must include 'mlb_player_id' and 'Date'
injury_stats = pd.read_sql_query('SELECT * from injury_stats', conn)  # must include 'mlb_player_id' and 'injury_date'

# STEP 2: Convert date columns
game_logs["Date"] = pd.to_datetime(game_logs["Date"])
injury_stats["injury_date"] = pd.to_datetime(injury_stats["injury_date"])

# STEP 3: Initialize all as not injured
game_logs["Injured"] = 0

# STEP 4: Iterate over each injury record
for _, row in injury_stats.iterrows():
    player_id = row["mlb_player_id"]
    injury_date = row["injury_date"]

    # Mark all games within 30 days before the injury as Injured = 1
    mask = (
        (game_logs["mlb_player_id"] == player_id) &
        (game_logs["Date"] <= injury_date) &
        (game_logs["Date"] >= injury_date - timedelta(days=30))
    )
    game_logs.loc[mask, "Injured"] = 1

game_logs.rename(columns={"mlb_player_id": "PlayerID"}, inplace=True)



In [67]:
game_logs.head(5)

,Name,PlayerID,Date,Team,OPP,GP,GS,INN,PA,NP,...,SLG,PrimaryPOS,Throw,Bat,Height,Weight,Age,BirthDate,Debut,Injured
0,Donovan Solano,456781,2024-05-06,San Diego Padres,Chicago Cubs,1,0.0,0,4.0,20.0,...,.667,First Base,Right,Right,"5' 8""",210,37,1987-12-17,2012-05-21,0
1,Donovan Solano,456781,2024-05-07,San Diego Padres,Chicago Cubs,1,0.0,0,4.0,15.0,...,.429,First Base,Right,Right,"5' 8""",210,37,1987-12-17,2012-05-21,0
2,Donovan Solano,456781,2024-05-11,San Diego Padres,Los Angeles Dodgers,1,0.0,0,4.0,26.0,...,.500,First Base,Right,Right,"5' 8""",210,37,1987-12-17,2012-05-21,0
3,Donovan Solano,456781,2024-05-15,San Diego Padres,Colorado Rockies,1,0.0,0,4.0,18.0,...,.538,First Base,Right,Right,"5' 8""",210,37,1987-12-17,2012-05-21,0
4,Donovan Solano,456781,2024-05-19,San Diego Padres,Atlanta Braves,1,0.0,0,0.0,0.0,...,.538,First Base,Right,Right,"5' 8""",210,37,1987-12-17,2012-05-21,0


## Model Training

In [79]:
injury_risk_model = '/Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr/mlb/models/injury_risk_model.pkl'

# Load trained model and feature names
xgb_model = joblib.load(injury_risk_model)
feature_names = xgb_model.get_booster().feature_names

# STEP 1: Load current player data (replace this with your data source)
# Example: new_game_logs = pd.read_csv('game_logs.csv')

new_game_logs = game_logs.copy()

# STEP 2: Preprocess data (same as training)
new_game_logs["Date"] = pd.to_datetime(new_game_logs["Date"])
new_game_logs = new_game_logs.drop(columns=["Name", "Date", "Team", "OPP"])

# Identify stat columns
stats_cols = new_game_logs.drop(columns=["PlayerID", "Injured"]).columns

new_game_logs_numeric = new_game_logs[stats_cols].apply(pd.to_numeric, errors='coerce')

# Prepare rolling DataFrames
roll5 = new_game_logs_numeric.groupby(new_game_logs["PlayerID"]).rolling(5, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll5')
roll10 = new_game_logs_numeric.groupby(new_game_logs["PlayerID"]).rolling(10, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll10')

# Concatenate once to avoid fragmentation
new_game_logs = pd.concat([new_game_logs, roll5, roll10], axis=1)

# Drop raw stats and player ID
new_game_logs = new_game_logs.drop(columns=list(stats_cols) + ["PlayerID"])
new_game_logs = new_game_logs.fillna(0)

# STEP 3: Filter for active, uninjured players
uninjured_players = new_game_logs[new_game_logs['Injured'] == 0]

# Drop label before prediction and ensure feature order matches training
X_uninjured = uninjured_players.drop(columns=['Injured'])
X_uninjured = X_uninjured[feature_names]

# STEP 4: Predict injury risk probabilities
injury_probs = xgb_model.predict_proba(X_uninjured)[:, 1]

# STEP 5: Add probabilities to DataFrame
uninjured_players = uninjured_players.copy()
uninjured_players['Injury_Risk'] = injury_probs

# STEP 6: Add back PlayerID and Name for clarity (from original game_logs)
uninjured_players['PlayerID'] = game_logs.loc[uninjured_players.index, 'PlayerID']
uninjured_players['Name'] = game_logs.loc[uninjured_players.index, 'Name']

# STEP 7: Rank players by predicted injury risk (highest to lowest)
ranked_risk = uninjured_players[['PlayerID', 'Name', 'Injury_Risk']]
ranked_risk = (
    uninjured_players[['PlayerID', 'Name', 'Injury_Risk']]
    .sort_values(by='Injury_Risk', ascending=False)
    .drop_duplicates(subset='PlayerID', keep='first')
)
# Add a formatted percentage column
ranked_risk['Injury_Risk_Percent'] = (ranked_risk['Injury_Risk'] * 100).round(2).astype(str) + '%'

## Injury List

In [82]:
# Top 30 current mlb players with highest injury risk

print(ranked_risk.head(30))

        PlayerID                 Name  Injury_Risk Injury_Risk_Percent
15672     663656          Kyle Tucker     0.986892              98.69%
30757     681082         Bryson Stott     0.985140              98.51%
10618     642715         Willy Adames     0.984421              98.44%
5208      600869    Jeimer Candelario     0.983700              98.37%
30164     680757          Steven Kwan     0.982322              98.23%
24304     669707         Jared Triolo     0.981270              98.13%
17759     665052       Griffin Conine     0.980574              98.06%
386086    687957        Dustin Harris     0.980572              98.06%
6969      608671     Travis Jankowski     0.978696              97.87%
17212     664761            Alec Bohm     0.978692              97.87%
19331     666134          Nolan Jones     0.978129              97.81%
24985     670764         Taylor Walls     0.978022               97.8%
9469      641313         Tim Anderson     0.976989               97.7%
34515 

In this analysis, we used an XGBoost classification model trained on MLB player game logs, incorporating rolling averages (over the last 5 and 10 games) of key workload, performance, and biometric features. For each active, uninjured player, the model predicts an injury risk probability between 0 and 1, which we convert into an easy-to-read percentage (e.g., 98.69%).

The Injury_Risk score reflects how closely a player’s recent workload and stats match patterns historically observed before injuries (within 30 days prior) in the training data. For example, high-risk players like Kyle Tucker and Bryson Stott show recent game trends (heavy workloads, high batting/fielding activity) that the model has learned to associate with elevated injury likelihood.

The ranking you see is sorted by predicted risk, with the top players showing metrics (e.g., high plate appearances, frequent starts, or accumulated fatigue) that the model interprets as increasing their odds of injury. Keep in mind, the precision and recall balance in the model is tuned to prioritize catching as many potential injuries as possible (high recall), even if it means some false positives (lower precision), which explains why even elite or healthy players might appear near the top.

## LA Dodgers Injury Risk

In [78]:
# Injury risk prediction for the Los Angeles Dodgers in 2025

dodgers_game_logs = game_logs[
    (game_logs['Team'] == 'Los Angeles Dodgers') & (game_logs['Date'].dt.year == 2025)
]

# Load trained model and feature names
injury_risk_model = '/Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr/mlb/models/injury_risk_model.pkl'
xgb_model = joblib.load(injury_risk_model)
feature_names = xgb_model.get_booster().feature_names

# Preprocess data
dodgers_new_game_logs = dodgers_game_logs.copy()
dodgers_new_game_logs = dodgers_new_game_logs.drop(columns=["Name", "Date", "Team", "OPP"])
dodgers_stats_cols = dodgers_new_game_logs.drop(columns=["PlayerID", "Injured"]).columns

# Calculate rolling averages
dodgers_new_game_logs_numeric = dodgers_new_game_logs[dodgers_stats_cols].apply(pd.to_numeric, errors='coerce')
dodgers_roll5 = (
    dodgers_new_game_logs_numeric.groupby(dodgers_new_game_logs["PlayerID"])
    .rolling(5, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll5')
)
dodgers_roll10 = (
    dodgers_new_game_logs_numeric.groupby(dodgers_new_game_logs["PlayerID"])
    .rolling(10, min_periods=1).mean().reset_index(level=0, drop=True).add_suffix('_roll10')
)

# Combine once to avoid fragmentation
dodgers_new_game_logs = pd.concat([dodgers_new_game_logs, dodgers_roll5, dodgers_roll10], axis=1)
dodgers_new_game_logs = dodgers_new_game_logs.drop(columns=list(dodgers_stats_cols) + ["PlayerID"])
dodgers_new_game_logs = dodgers_new_game_logs.fillna(0)

# Filter active, uninjured players
dodgers_uninjured_players = dodgers_new_game_logs[dodgers_new_game_logs['Injured'] == 0]

# Prepare data for prediction
X_dodgers_uninjured = dodgers_uninjured_players.drop(columns=['Injured'])
X_dodgers_uninjured = X_dodgers_uninjured[feature_names]

# Predict injury risk
dodgers_injury_probs = xgb_model.predict_proba(X_dodgers_uninjured)[:, 1]

# Add predictions
dodgers_uninjured_players = dodgers_uninjured_players.copy()
dodgers_uninjured_players['Injury_Risk'] = dodgers_injury_probs

# Add back PlayerID and Name
dodgers_uninjured_players['PlayerID'] = game_logs.loc[dodgers_uninjured_players.index, 'PlayerID']
dodgers_uninjured_players['Name'] = game_logs.loc[dodgers_uninjured_players.index, 'Name']

# Rank players by risk (keep only highest per player)
dodgers_ranked_risk = (
    dodgers_uninjured_players[['PlayerID', 'Name', 'Injury_Risk']]
    .sort_values(by='Injury_Risk', ascending=False)
    .drop_duplicates(subset=['PlayerID'], keep='first')
    .reset_index(drop=True)
)
# Add a formatted percentage column
dodgers_ranked_risk['Injury_Risk_Percent'] = (dodgers_ranked_risk['Injury_Risk'] * 100).round(2).astype(str) + '%'


In [ ]:
#top 10 current dodgers players with highest injury risk

print(dodgers_ranked_risk.head(10))

   PlayerID               Name  Injury_Risk Injury_Risk_Percent
0    571970          Max Muncy     0.929502              92.95%
1    687221     Dalton Rushing     0.895328              89.53%
2    681546       James Outman     0.846650              84.66%
3    808975       Hyeseong Kim     0.819206              81.92%
4    621035       Chris Taylor     0.809580              80.96%
5    592696      Eddie Rosario     0.798012               79.8%
6    606192  Teoscar Hernández     0.781091              78.11%
7    518692    Freddie Freeman     0.770524              77.05%
8    676439    Hunter Feduccia     0.646990               64.7%
9    605141       Mookie Betts     0.643853              64.39%
